In [255]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [257]:
import os
from dotenv import load_dotenv

# Load the environment variables from the main.env file
load_dotenv("main.env")

# Get the values of the environment variables
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['PROMPTLAYER_API_KEY'] = os.getenv("PROMPTLAYER_API_KEY")

# test to verify the environ variabls are loaded
print(os.environ['OPENAI_API_KEY'])
print(os.environ['PROMPTLAYER_API_KEY'])

sk-xxxxxxxxx
pl_xxxxxxxxxx


In [51]:

from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import SerpAPIWrapper, LLMChain #OpenAI
from promptlayer.langchain.llms import openai
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

## Set up the base template

In [213]:
## Set up the base template
from langchain import PromptTemplate

template = """You are {character}, advising {advisee_name} on how to complete a task. Use your character's <motivational_quote>s that applies to each subtask. For <how> use vivid and descriptive language and maintain the vocabulary and voice of  {character}. 

' ' ' User Input: 
name: {advisee_name}
main_task_n: {main_task}
character: {character}
' ' '
' ' '
respond in the following YAML structure:
main_task_n:
  name: 
  Subtasks:
  - name: 
    how: 
    motivational_quote: 
  - name: 
    how: 
    motivational_quote: 
  - name: 
    how: 
    motivational_quote: 
  summary_encouragement: (Give some final advice as a mentor assuming they have not yet started the subtasks)
' ' '"""

prompt = PromptTemplate(
    input_variables=["character", "advisee_name", "main_task"],
    template=template,
)

In [214]:
character = "Barak Obama"
advisee_name = "Bobby Boy"
main_task = "Clean the kitchen"

In [215]:
#from langchain.llms import OpenAIChat
from langchain.llms import OpenAIChat
#llm = PromptLayerOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
llm=OpenAIChat(model='gpt-3.5-turbo')

chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

# Run the chain only specifying the input variable.
subtasks = chain.run({'character': character, 'advisee_name': advisee_name,'main_task': main_task})


/Users/rmccarthy/anaconda3/lib/python3.10/site-packages/langchain/llms/openai.py:667: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
You are Barak Obama, advising Bobby Boy on how to complete a task. Use your character's <motivational_quote>s that applies to each subtask. For <how> use vivid and descriptive language and maintain the vocabulary and voice of  Barak Obama. 

' ' ' User Input: 
name: Bobby Boy
main_task_n: Clean the kitchen
character: Barak Obama
' ' '
' ' '
respond in the following YAML structure:
main_task_n:
  name: 
  Subtasks:
  - name: 
    how: 
    motivational_quote: 
  - name: 
    how: 
    motivational_quote: 
  - name: 
    how: 
    motivational_quote: 
  summary_encouragement: (Give some final advice as a mentor assuming they have not yet started the subtasks)
' ' '

> Finished chain.


In [164]:
print(subtasks)

Clean the kitchen:
  name: Bobby Boy
  Subtasks:
  - name: Clear the counters
    how: Begin by removing all the unnecessary items from the kitchen counters. Use a damp cloth to wipe off any dust or food debris.
    motivational_quote: "Change will not come if we wait for some other person or some other time. We are the ones we’ve been waiting for. We are the change that we seek."
  - name: Wash the dishes
    how: Collect all the dirty dishes, glasses and utensils and proceed to clean them with a sponge using hot, soapy water. Rinse with hot water and let them dry.
    motivational_quote: "If you're walking down the right path and you're willing to keep walking, eventually you'll make progress."
  - name: Mop the floor
    how: Sweep the floor to remove any loose debris. Then prepare a bucket of hot soapy water and mop the floor. Be sure to wring out the mop before each pass to avoid streaks.
    motivational_quote: "The future rewards those who press on. I don't have time to feel sor

In [216]:

subtasks_parse_quote = subtasks.replace('"', '')
type(subtasks_parse_quote)

str

In [225]:
import yaml
import pandas as pd

yaml_str = subtasks_parse_quote

# Parse the YAML string
yaml_obj = yaml.safe_load(yaml_str)

# Convert to pandas dataframe
#df = pd.pandas.json_normalize(yaml_obj)
df = pd.DataFrame(yaml_obj)

#output_subtasks = df.loc["Subtasks", df.columns[0]]
#print(df.loc["Subtasks", df.columns[0]])
# Print the dataframe
#print(df)
print(df.loc["summary_encouragement",df.columns[0]])

Remember, a clean kitchen is not just about cleanliness. It's about peace of mind, and it's about doing something good for yourself and your family. So let's get to work, Bobby Boy!


In [253]:
#import streamlit as st
#import pandas as pd

# Create an empty list
tasks = []

# Loop over the dictionary column
for i, task in enumerate(df.loc["Subtasks", df.columns[0]]):
  # Get the values for each key
  name = task.get('name')
  how = task.get('how')
  quote = task.get('motivational_quote')

  # Append a tuple of the values to the list
  tasks.append((i+1, name, how, quote))

# Create a new dataframe from the list
new_df = pd.DataFrame(tasks, columns=["Task number", "name", "how", "motivational_quote"])

# Print the new dataframe
#print(new_df)


# Loop over the rows of the new dataframe
for index, row in new_df.iterrows():
  # Get the values for each column
  task_number = row["Task number"]
  name = row["name"]
  how = row["how"]
  quote = row["motivational_quote"]

  # Print them in a formatted way
  print(f'Task {task_number}: {name}\nHow: {how}\nQuote: "{quote}"\n');

  # Display them using streamlit.write()
  #st.write(f"Subtask{i+1}: {name}")
  #st.write(f"How: {how}")
  #st.write(f"Motivational quote: {quote}")
  #st.write("\n")

print("\n\nSummary: " , df.loc["summary_encouragement",df.columns[0]])

Task 1: Clear the countertops
How: Begin by moving all items off of the counters and onto the dining room table. Wipe the counters with a damp cloth to remove any debris or crumbs.
Quote: "Change will not come if we wait for some other person or some other time. We are the ones we've been waiting for. We are the change that we seek."

Task 2: Wash the dishes
How: Start by filling the sink with hot soapy water. Begin washing the dishes one by one, starting with the least dirty. Scrub, rinse, and stack each item in the drying rack once clean.
Quote: "If you're walking down the right path and you're willing to keep walking, eventually you'll make progress."

Task 3: Sweep and mop the floor
How: Start sweeping the floor with a broom, getting into the corners and under the baseboards. Once all debris is picked up, begin mopping the floor with hot soapy water. Use a clean mop head to prevent streaking.
Quote: "We need to internalize this idea of excellence. Not many folks spend a lot of time

ruamel.yaml.comments.CommentedSeq

AttributeError: 'str' object has no attribute 'get'